In [1]:
import pandas as pd
import numpy as np
import datetime
import imp
import scripts as scr
import os
import re
import multiprocessing
import numpy as np
import pandas as pd
from pytictoc import TicToc
from IPython.display import display
import itertools
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm_notebook
import os

In [15]:
from whois import whois

In [2]:
FIRST_N = 100
N_THREADS = 16
train_folder = '../../data/raw_splits/train/'
train_files = sorted([x for x in os.listdir(train_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
train_files = [os.path.join(train_folder, x) for x in train_files]
print(f'Length of train files is {len(train_files)}')
train_files[:5]

Length of train files is 51


['../../data/raw_splits/train/chunk_0.fth',
 '../../data/raw_splits/train/chunk_1.fth',
 '../../data/raw_splits/train/chunk_2.fth',
 '../../data/raw_splits/train/chunk_3.fth',
 '../../data/raw_splits/train/chunk_4.fth']

In [3]:
test_folder = '../../data/raw_splits/test/'
test_files = sorted([x for x in os.listdir(test_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
test_files = [os.path.join(test_folder, x) for x in test_files]
print(f'Length of test files is {len(test_files)}')
test_files[:5]

Length of test files is 26


['../../data/raw_splits/test/chunk_0.fth',
 '../../data/raw_splits/test/chunk_1.fth',
 '../../data/raw_splits/test/chunk_2.fth',
 '../../data/raw_splits/test/chunk_3.fth',
 '../../data/raw_splits/test/chunk_4.fth']

In [4]:
files = train_files + test_files

In [5]:
def get_data(filename):
    df = pd.read_feather(filename)
    df = df[['ip_address', 'ip_country', 'ip_region', 'ip_city', 'ip_isp']]
    df = df.loc[df['ip_address'] != 'null'].drop_duplicates().copy()
    return df

In [6]:
%%time
# жрет вплоть более 70 гигов оперативки, осторожнее
with multiprocessing.Pool(processes=min(N_THREADS, FIRST_N)) as pool:
    results = pool.map(get_data, files)

CPU times: user 5.49 s, sys: 2.31 s, total: 7.8 s
Wall time: 1min 18s


In [7]:
df = pd.concat(results, ignore_index = True).drop_duplicates()

In [13]:
(df['ip_country'] != 'ru').sum()

184772

In [9]:
df['ip_country'].value_counts()

ru    2550606
ua      26044
by      15688
th       9240
us       9184
cn       8292
kz       8125
tr       7061
de       6736
it       6674
es       5761
am       5617
az       4360
++       4099
nl       3526
gb       3202
fr       3089
gr       2883
??       2731
cz       2715
ge       2416
in       2199
vn       2061
kg       2029
il       1920
eg       1914
kr       1896
md       1820
id       1800
ae       1760
       ...   
gp          3
fj          3
lc          3
mq          3
na          2
bo          2
im          2
cm          2
ax          2
cw          2
li          1
ne          1
mp          1
mf          1
a2          1
bb          1
gi          1
vi          1
tl          1
zw          1
pr          1
bm          1
gn          1
jm          1
et          1
uy          1
mg          1
bw          1
gq          1
gt          1
Name: ip_country, Length: 172, dtype: int64

основная идея - берем данные whois для ip адреса и смотрим есть ли надписи мтс мегафон и тд
чтобы уменьшить перебор возьмем только первые два числа ip адреса

In [16]:
pseudo_unique_ips = {}
for ip in sorted([x for x in df['ip_address'].unique() if str(x).count('.') == 3]):
    subnet = ip.rsplit('.', 2)[0]
    if subnet in pseudo_unique_ips:
        continue
    else:
        pseudo_unique_ips[subnet] = ip

In [17]:
len(pseudo_unique_ips)

11139

In [18]:
ip

'99.91.9.153'

In [22]:
ip_results = {}
bad_results = {}
for subnet, ip in tqdm_notebook(pseudo_unique_ips.items(), total = len(pseudo_unique_ips)):
    try:
        ip_results[subnet] = os.popen(f'whois {ip}').readlines()
    except:
        bad_results[subnet] = ip

HBox(children=(IntProgress(value=0, max=11139), HTML(value='')))

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



---
посмотрим на результаты

In [32]:
big_list = list(itertools.chain.from_iterable([''.join(x).lower().replace('\n', '').split(' ') for x in ip_results.values()]))

In [30]:
pd.Series(big_list).value_counts()

                                      3474703
-                                       35683
RIPE                                    32428
to                                      32097
the                                     31449
is                                      22649
for                                     18580
Database                                18559
Information                             17613
related                                 16571
This                                    15980
#                                       15728
The                                     15017
query                                   14800
and                                     14290
output                                  12904
contact                                 12339
in                                      11503
Abuse                                   10079
are                                     10021
subject                                  9911
Terms                             

In [33]:
s = set(big_list)

In [77]:
'mts' in s, 'megafon' in s, 'tele2' in s, 'mobile' in s

(True, True, True, True)

In [57]:
m = [y for y in [''.join(x).lower() for x in ip_results.values()] if y.count('vimpelcom')]

In [58]:
len(m)

20

In [59]:
print(m[10])

% this is the ripe database query service.
% the objects are in rpsl format.
%
% the ripe database is subject to terms and conditions.
% see http://www.ripe.net/db/support/db-terms-conditions.pdf

% note: this output has been filtered.
%       to receive output for a database update, use the "-b" flag.

% information related to '62.141.100.0 - 62.141.100.255'

% abuse contact for '62.141.100.0 - 62.141.100.255' is 'abuse-b2b@beeline.ru'

inetnum:        62.141.100.0 - 62.141.100.255
netname:        ru-combellga-p2p-et-customer-ip-pool-net
descr:          111250 russia moscow sovintel/edn et combellga
descr:          clients static ip pool (xdsl, xeth, p2p, etc ...)
status:         assigned pa
country:        ru
admin-c:        svnt1-ripe
tech-c:         svnt2-ripe
mnt-by:         sovintel-mnt
remarks:        +-------------------------------------------+
remarks:        | please send abuse to abuse@b2b.beeline.ru |
remarks:        +-------------------------------------------+
created:  

теперь пройдемся по прямому вхождению ключевых слов

In [74]:
def parse_ip_results(tlist):
    tset = set(''.join(tlist).replace('\n', '').lower().split(' '))
    is_mts, is_beeline, is_megafon, is_tele2, is_ru_mobile = 0, 0, 0, 0, 0
    if 'mts' in tset:
        is_mts = 1
    if ('beeline' in tset) or ('vimpelcom' in tset):
        is_beeline = 1
    if 'megafon' in tset:
        is_megafon = 1
    if 'tele2' in tset:
        is_tele2 = 1
    is_ru_mobile = 1 if any((is_mts, is_beeline, is_megafon, is_tele2)) else 0
    return is_mts, is_beeline, is_megafon, is_tele2, is_ru_mobile

In [76]:
parsed_df = pd.DataFrame([(subnet, *parse_ip_results(ip_info)) for subnet, ip_info in ip_results.items()],
             columns = ['subnet', 'is_mts', 'is_beeline', 'is_megafon', 'is_tele2', 'is_ru_mobile'])

In [79]:
parsed_df.head(10)

,subnet,is_mts,is_beeline,is_megafon,is_tele2,is_ru_mobile
0,1.0,0,0,0,0,0
1,1.1,0,0,0,0,0
2,1.10,0,0,0,0,0
3,1.102,0,0,0,0,0
4,1.11,0,0,0,0,0
5,1.127,0,0,0,0,0
6,1.136,0,0,0,0,0
7,1.144,0,0,0,0,0
8,1.152,0,0,0,0,0
9,1.163,0,0,0,0,0


In [86]:
df = df[df['ip_address'].notnull()]

In [87]:
df['subnet'] = df['ip_address'].apply(lambda x: x.rsplit('.', 2)[0])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [94]:
parsed_df['subnet'].nunique()

10935

In [95]:
parsed_df['subnet'].count()

10935

In [98]:
df.loc[df['ip_address'] == '5.141.198.167']

,ip_address,ip_country,ip_region,ip_city,ip_isp,subnet
1413415,5.141.198.167,ru,13,Chelyabinsk,4697132,5.141
3234843,5.141.198.167,ru,13,Chelyabinsk,7042071,5.141
4532536,5.141.198.167,ru,13,Zlatoust,7042071,5.141
7978063,5.141.198.167,ru,13,Zlatoust,7.04207e+06,5.141
7978064,5.141.198.167,ru,13,Chelyabinsk,7.04207e+06,5.141


In [97]:
df['ip_address'].value_counts()

5.141.198.167      5
213.151.5.225      5
80.83.237.40       5
95.153.132.153     5
95.153.129.49      5
95.153.133.150     5
85.140.3.135       5
176.59.194.166     5
95.153.132.168     5
85.140.2.113       5
85.140.3.228       5
85.26.186.37       5
217.118.83.183     5
85.140.7.147       5
176.59.85.228      5
188.170.192.54     5
95.153.128.243     5
85.26.186.162      5
188.162.163.222    5
85.140.3.48        5
176.59.107.42      5
95.153.134.51      5
95.153.130.229     5
176.59.105.110     5
85.140.3.182       5
85.140.3.19        5
85.140.3.235       5
95.153.129.76      5
95.153.129.12      5
95.153.133.151     5
                  ..
81.5.103.203       1
95.52.215.60       1
178.167.25.112     1
178.217.60.201     1
109.196.75.218     1
178.186.37.104     1
95.54.237.126      1
109.165.43.179     1
78.157.248.101     1
91.105.184.152     1
178.204.88.118     1
176.114.190.251    1
95.106.15.248      1
188.68.217.188     1
95.52.63.42        1
212.22.90.140      1
94.50.207.12 

In [99]:
df_merged = df[['ip_address', 'subnet']].drop_duplicates().merge(parsed_df, how = 'left', on = 'subnet')
df_merged.loc[:, ['is_mts', 'is_beeline', 'is_megafon', 'is_tele2', 'is_ru_mobile']] = df_merged.loc[:, ['is_mts', 'is_beeline', 'is_megafon', 'is_tele2', 'is_ru_mobile']].fillna(0).astype(int)

In [100]:
df_merged

,ip_address,subnet,is_mts,is_beeline,is_megafon,is_tele2,is_ru_mobile
0,31.173.86.8,31.173,0,0,1,0,1
1,31.173.82.117,31.173,0,0,1,0,1
2,31.173.85.200,31.173,0,0,1,0,1
3,31.173.82.110,31.173,0,0,1,0,1
4,31.173.81.68,31.173,0,0,1,0,1
5,31.173.81.24,31.173,0,0,1,0,1
6,31.173.86.3,31.173,0,0,1,0,1
7,31.173.86.248,31.173,0,0,1,0,1
8,213.87.158.130,213.87,1,0,0,0,1
9,31.173.86.150,31.173,0,0,1,0,1


In [105]:
df_merged.shape, df_merged['ip_address'].nunique()

((2301678, 7), 2301678)

In [104]:
df_merged['subnet'].count(), df_merged['subnet'].nunique() - len(bad_results)

(2301678, 10935)

In [107]:
df_merged.drop('subnet', axis = 1).to_pickle('../../data/ip_info.pkl')